In [30]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")


In [2]:
import torch
from transformers import BitsAndBytesConfig, AutoModel

bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

encoder = AutoModel.from_pretrained(
'Salesforce/SFR-Embedding-Mistral',
trust_remote_code=True,
device_map='auto',
torch_dtype=torch.bfloat16,
quantization_config=bnb_config
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
#embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
#embed_model_id = 'voyage-lite-02-instruct' #not available through HF
embed_model_id = 'Salesforce/SFR-Embedding-Mistral'
core_embeddings_model = HuggingFaceEmbeddings(
    model_name=encoder,
    model_kwargs={"trust_remote_code":True}
    # model_name=embed_model_id
)
embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id

)

ValidationError: 1 validation error for HuggingFaceEmbeddings
model_name
  str type expected (type=type_error.str)

In [31]:
#embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
#embed_model_id = 'voyage-lite-02-instruct' #not available through HF
#embed_model_id = 'WhereIsAI/UAE-Large-V1'
embed_model_id = 'mixedbread-ai/mxbai-embed-large-v1'
core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={"trust_remote_code":True}
    # model_name=embed_model_id
)
embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id

)

In [4]:
import os
existing_files = set(os.listdir('../abstracts'))

In [29]:
#existing_files = list(existing_files)

In [10]:
def get_title_abstract_filename(directory = '../abstracts/',filename=None):
    path = os.path.join(directory, filename)
    with open(path, 'r') as file:
        lines = file.readlines()
        title = lines[0].strip()
        abstract = "".join(lines[1:]).replace('\n', ' ')
    return title, abstract, filename


In [11]:
get_title_abstract_filename(filename=next(iter(existing_files)))

('Language Models are Few-Shot Learners',
 "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something which current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches. Specifically, we train GPT-3, an autoregressive language model with 175 billion parameters, 10x more than any previous non-sparse language model, and test its performance in the few-shot setting. For all tasks, GPT-3 is applied without any gradient updates or 

In [13]:
for file in existing_files:
    title, abstract,_ = get_title_abstract_filename(filename=file)
    print(title)
    print(abstract)
    print()

Language Models are Few-Shot Learners
Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something which current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches. Specifically, we train GPT-3, an autoregressive language model with 175 billion parameters, 10x more than any previous non-sparse language model, and test its performance in the few-shot setting. For all tasks, GPT-3 is applied without any gradient updates or fine-t

In [9]:
from langchain.schema.document import Document

In [34]:
docs

[Document(page_content='Generation of plausible yet incorrect factual information, termed hallucination, is an unsolved issue in large language models. We study the ability of language models to deliberate on the responses they give in order to correct their mistakes. We develop the Chain-of-Verification (CoVe) method whereby the model first (i) drafts an initial response; then (ii) plans verification questions to fact-check its draft; (iii) answers those questions independently so the answers are not biased by other responses; and (iv) generates its final verified response. In experiments, we show CoVe decreases hallucinations across a variety of tasks, from list-based questions from Wikidata, closed book MultiSpanQA and longform text generation.', metadata={'source': 'Chain-of-Verification Reduces Hallucination in Large Language Models'}),
 Document(page_content='This work aims at decreasing the end-to-end generation latency of large language models (LLMs). One of the major causes of

In [48]:
docs = [get_title_abstract_filename(filename=file) for file in existing_files]
docs = [Document(page_content=doc[1],metadata={'source':doc[0], 'filename':doc[2]}) for doc in docs]


In [49]:
len(docs)

37

In [15]:
dir(docs[0])

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__validators__',
 '_abc_impl',
 '_calculate_keys',
 '_copy_and_set_values',
 '_decompose_class',
 '_enfo

In [16]:
docs[0].__getattribute__('page_content')

"Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something which current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches. Specifically, we train GPT-3, an autoregressive language model with 175 billion parameters, 10x more than any previous non-sparse language model, and test its performance in the few-shot setting. For all tasks, GPT-3 is applied without any gradient updates or fine-tuning, with tasks and few-shot demons

In [50]:
len(docs)

37

In [51]:
for i, doc in enumerate(docs):
   #doc = doc[1]
   #content = docs[index]
   if i == 0:
       vector_store = FAISS.from_documents([doc], embedder)
   else:
      vector_store_i = FAISS.from_documents([doc], embedder)
      vector_store.merge_from(vector_store_i)

vector_store

In [59]:
dir(vector_store)

['_FAISS__add',
 '_FAISS__from',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_aembed_documents',
 '_aembed_query',
 '_asimilarity_search_with_relevance_scores',
 '_cosine_relevance_score_fn',
 '_create_filter_func',
 '_embed_documents',
 '_embed_query',
 '_euclidean_relevance_score_fn',
 '_get_retriever_tags',
 '_max_inner_product_relevance_score_fn',
 '_normalize_L2',
 '_select_relevance_score_fn',
 '_similarity_search_with_relevance_scores',
 'aadd_documents',
 'aadd_texts',
 'add_documents',
 'add_embeddings',
 'add_texts',
 'adelete',
 'afrom_documents',
 'afrom_embeddings',
 'afrom_texts',
 

In [52]:
vector_store.index_to_docstore_id

{0: '7a897c10-4cb2-4513-b9b2-54899b4ba418',
 1: 'a7749943-9b5c-4e4c-ba77-34ec4759d4f5',
 2: '77bbd07a-a9dc-42a2-9ecb-18ef8ac784c9',
 3: 'e353bcb8-b415-4bdc-8598-f2aeb26073fb',
 4: '008c2ed5-ab6d-4a70-a280-6b9a12ae7485',
 5: '4062bce4-e712-4e0f-ba39-abf0c7ecc8b5',
 6: '75040030-3c16-4a19-85bf-ec3fe7f73b4e',
 7: 'bc16ac76-0a88-4ff6-98f4-907912ab7bd4',
 8: 'b0e2d04a-c0b3-4bcf-8213-7fd11c72ee45',
 9: 'fd5420b4-6e24-411a-acdd-293a4f9a107e',
 10: '2a394c86-d805-4d29-879f-28276b841495',
 11: 'fdb249fa-d067-4168-99b4-bbaba44a672d',
 12: 'f9146fc1-64e4-47ff-a750-f71b62b1d761',
 13: 'a45b338b-170c-4f1f-a9a2-24868687edbf',
 14: 'e6c645da-ee4d-4ec9-b40b-02d58506e5ab',
 15: 'cf77757b-1720-4864-a6b9-dd31c488c6c8',
 16: 'a22b0f34-947d-48e4-9df6-fff2991c4971',
 17: 'babddf37-c6f7-4475-9e51-ef9ae594e07b',
 18: 'ac0dd0fd-376a-4403-a809-1471ded96fbf',
 19: '3337f6b1-0cf9-44f1-a2ad-e1b458c937b0',
 20: 'fa7e50fd-5a38-4ecf-99b6-00913db308bd',
 21: '6f27c859-b980-4737-b421-7fc54dd9a03f',
 22: '81d0e35d-b742-

In [53]:
#vector_store.asimilaritysearch('What is PPO?')
vector_store.asimilarity_search('What is PPO?')

<coroutine object FAISS.asimilarity_search at 0x7809be41d480>

In [58]:
query = "The complexity of the alignment problem stems from the fact that existing methods are unstable. Researchers continuously invent various tricks to address this shortcoming. For instance, in the fundamental Reinforcement Learning From Human Feedback (RLHF) technique of Language Model alignment, in addition to reward maximization, the Kullback-Leibler divergence between the trainable policy and the SFT policy is minimized. This addition prevents the model from being overfitted to the Reward Model (RM) and generating texts that are out-of-domain for the RM. The Direct Preference Optimization (DPO) method reformulates the optimization task of RLHF and eliminates the Reward Model while tacitly maintaining the requirement for the policy to be close to the SFT policy. In our paper, we argue that this implicit limitation in the DPO method leads to sub-optimal results. We propose a new method called Trust Region DPO (TR-DPO), which updates the reference policy during training. With such a straightforward update, we demonstrate the effectiveness of TR-DPO against DPO on the Anthropic HH and TLDR datasets. We show that TR-DPO outperforms DPO by up to 19%, measured by automatic evaluation with GPT-4. The new alignment approach that we propose allows us to improve the quality of models across several parameters at once, such as coherence, correctness, level of detail, helpfulness, and harmlessness."
#query = 'The quadratic complexity and weak length extrapolation of Transformers limits their ability to scale to long sequences, and while sub-quadratic solutions like linear attention and state space models exist, they empirically underperform Transformers in pretraining efficiency and downstream task accuracy. We introduce Megalodon, a neural architecture for efficient sequence modeling with unlimited context length. Megalodon inherits the architecture of Mega (exponential moving average with gated attention), and further introduces multiple technical components to improve its capability and stability, including complex exponential moving average (CEMA), timestep normalization layer, normalized attention mechanism and pre-norm with two-hop residual configuration. In a controlled head-to-head comparison with Llama2, Megalodon achieves better efficiency than Transformer in the scale of 7 billion parameters and 2 trillion training tokens. Megalodon reaches a training loss of 1.70, landing mid-way between Llama2-7B (1.75) and 13B (1.67). Code: https://github.com/XuezheMax/megalodon'
#query = ''
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)
  print(page.metadata)

While large-scale unsupervised language models (LMs) learn broad world knowledge and some reasoning skills, achieving precise control of their behavior is difficult due to the completely unsupervised nature of their training. Existing methods for gaining such steerability collect human labels of the relative quality of model generations and fine-tune the unsupervised LM to align with these preferences, often with reinforcement learning from human feedback (RLHF). However, RLHF is a complex and often unstable procedure, first fitting a reward model that reflects the human preferences, and then fine-tuning the large unsupervised LM using reinforcement learning to maximize this estimated reward without drifting too far from the original model. In this paper we introduce a new parameterization of the reward model in RLHF that enables extraction of the corresponding optimal policy in closed form, allowing us to solve the standard RLHF problem with only a simple classification loss. The resu

In [35]:
# for i, doc in enumerate(docs):
#    #content = docs[index]
#    if i == 0:
#        vector_store = FAISS.from_documents(doc.__getattribute__('page_content'), embedder)
#    else:
#       vector_store_i = FAISS.from_documents(doc.__getattribute__('page_content'), embedder)
#       vector_store.merge_from(vector_store_i)

# vector_store

In [36]:
# for index, doc in enumerate(docs):
#    #content = docs[index]
#    if index == 0:
#        vector_store = FAISS.from_texts(doc.page_content, embedder)
#    else:
#       vector_store_i = FAISS.from_texts(doc.page_content, embedder)
#       vector_store.merge_from(vector_store_i)

# vector_store